In [1]:
# 교차검증과 그리드 서치

In [2]:
# 훈련세트로 모델을 훈련하고 테스트 세트에서 모델을 평가 -> 결국에는 테스트세트로도 훈련이 되는 꼴
# 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다. 모델을 만들고서 마지막에만 사용해야 합니다.
# 그렇다면 매개변수, 특히 하이퍼파라미터는 어떻게 조정해야 되나요? 
# -> 검증세트를 사용합니다.
# 훈련세트에서 20% 정도를 다시 검증세트로 할당하는 방법
# 훈련세트에서 모델을 훈련하고 검증 세트로 모델을 평가 (매개변수 튜닝, 하이퍼파라미터)
# 최적화된 매개변수를 갖고 훈련세트와 검증세트를 합쳐 전체 훈현 데이터에서 모델을 다시 훈련
# 마지막으로 테스트세트로 모델의 성능을 검증한다.

In [3]:
# 검증세트 만들기

In [4]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [5]:
# 클래스 열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장

In [6]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [7]:
# 테스트 세트와 훈련세트를 나누어 준다.

In [8]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [9]:
# 훈련세트의 일부분(20%)를 다시 검증세트로 만든다.

In [10]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [11]:
# 훈련세트와 검증세트의 크기 확인

In [12]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [13]:
# 5197의 20%인 1040가 검증세트로 할당되었고, 특성 3개를 사용하는 것을 확인 할 수 있다.

In [14]:
# 이제 훈련세트와 검증세트를 갖고 모델을 생성 및 평가

In [15]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [16]:
# 훈련세트에 과대적합되어있다. -> 매개변수를 바꿔서 더 좋은 모델을 찾아야한다.

In [17]:
# 교차 검증: 검증세트를 만드느라 훈련세트가 줄어든다. 검증 세트를 떼어내어 평가하는 과정을 여러번 반복하고 이 점수를 평균하여 최종 검증 점수를 얻는 방식
# -> 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다. * 노션 참고

In [18]:
# 사이킷런에는 cross_validate() 교차 검증 함수가 존재.
# 평가할 모델 객체를 첫 번째 매개변수로 전달, 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate()함수에 전달
# 그러면 알아서 훈련세트에서 검증세트를 떼어서 검증세트로 점수를 제시
# 만약 5-폴드 교차 검증(기본적으로)이면 5개의 검증세트 점수를 주겠지?

In [19]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01746392, 0.00773001, 0.00665808, 0.00570083, 0.00515509]), 'score_time': array([0.00140119, 0.00057507, 0.00045705, 0.00040913, 0.00040817]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [20]:
# fit_time score_time test_score 에 대한 키를 가진 딕셔너리를 반환 처음 두개는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미

In [21]:
# 나온 테스트스코어를 평균한 값이 교차검증의 최종 점수이다.

In [22]:
import numpy as np

print(np.mean(scores['test_score']))

0.8554925223957948


In [23]:
# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야합니다.
# 회귀 모델일 경우 Kfold 분할기
# 분류 모델일 경우 StratifiedKFold 분할기를 사용 (타깃 클래스를 골고루 나누기 위함)

In [24]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8554925223957948


In [25]:
# 어차피 기본적으로 같은 모델이 적용되어 있기 때문에 같은 값이 출력되는 것을 확인할 수 있다.

In [26]:
# 기본 5-폴드가 아닌 10-폴드 교차 검증을 수행하려면 다음과 같이 진행

In [27]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8581873425226026


In [28]:
# K 폴드도 같은 방식으로 사용 가능

In [29]:
# 파라미터 튜닝: 사용자가 지정해야만 하는 파라미터 (클래스나 메서드의 매개변수로 표현)
# 1. 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
# 2. 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꾸어본다.
# 3. 모델마다 매개변수를 바꿔가면서 모델을 훈련하고 교차 검증을 수행 해야한다.

# 그러나 하나의 파라미터를 고정하고 다른 파라미터를 변경하면 최적의 값을 찾을수 있을까?
# 답은 아니다. 하나의 파라미터 값이 바뀌면 다른 파라미터들도 영향을 받기 때문
# 매개변수가 많아지면 더더더더더 복잡하겠지?

# -> 사이킷런에서 제공하는 그리드서치를 사용한다.
# 하이퍼파라미터 탐색과 교차 검증을 한번에 수행 (별도로 cross_validate() 호출 x )

In [30]:
# 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만들어준다.

In [31]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [32]:
# 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만들어준다.

In [33]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [34]:
# 일반 모델을 훈련하는 것처럼 gs 객체에 fit()메소드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_inpurity_decrease 값을 바꿔가며 총 5번 실행
# 근데 교차 검증이 5-폴드 이기때문에 하나의 매개변수로 5번의 교차검증 즉, 5x5=25개의 모델을 훈련한다. 
# 많은 모델을 훈련하므로 n_jobs를 -1로 두어 시스템에 있는 모든 코어를 사용하도록 한다. (병렬 실행에 사용할 cpu 코어수 지정)

In [35]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [36]:
# 최적의 하이퍼 파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야하지!
# 그런데 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 모델을 훈련
# 최적으로 훈련된 모델은 gs 객체의 best_estimator_ 속성에 저장 -> 일반 경정 트리처럼 사용할 수 있다.

In [37]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [38]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장

In [39]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [40]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어 있다. 5번의 교차 검증으로 얻은 점수 출력

In [41]:
print(gs.cv_results_['mean_test_score'])

[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [42]:
# 각각의 매개변수에서의 교차검증 결과의 평균값 0.0001에서 가장 좋은 결과인 것을 확인 할 수 있다.

In [43]:
# 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인데스를 추출할 수 있다. 이후 해당 인덱스를 갖는 params 키에 저장된 매개변수 출력하면 된다.
# 그러면 0.0001 나오겠지. 앞서 진행한 best_params_ 속성과 같은 맥락

In [44]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [47]:
# 1. 먼저 탐색할 매개변수를 지정
# 2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 조합은 그리드 서치 객체에 저장된다.
# 3. 그리드 서치는 알아서 최상의 매개변수에서 전체 훈련세트를 사용해 최종 모델을 훈련한다. 해당 모델도 그리드 서치 객체에 저장된다.

In [48]:
# 결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정합니다.
# max_depth로 트리의 깊이를 제한
# min_samples_split로 노드를 나누기 위한 최소 샘플 수를 골라보자
# 즉, 매개변수 3개를 최적의 매개변수를 찾아보겠다!! 그리드서치를 사용해서!!

In [49]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [50]:
# 넘파이 arrange() 함수는 첫번째를 두번째까지 세번째 간격으로 증가시키는 배열
# range() 함수도 비슷하지만 정수만 사용할 수 있다.

# 위와 같은 경우에는 9x15x10 =1350개이다. 기본 5폴드 교차이므로 5를 곱하면 6750개이다.

In [51]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [52]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [53]:
# 최상의 매개변수 조합을 확인 할 수 있다.
# 최상의 교차 검증 점수를 확인해 보자.

In [54]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [55]:
# 'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12 매개변수를 사용하여 5-폴드 교차 검증의 평균값

In [56]:
# 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 그리드서치 클래스를 사용하여 원하는 매개변수 값을 나열하면 자동으로 교차검증으로 최적의 매개변수를 찾을 수 있다.
# 그러나 앞에서 탐색할 매개변수의 간격을 설정하였는데 특별한 근거는 존재하지 않는다.

# -> 랜덤 서치

In [57]:
# 랜덤서치: 매개변수 값의 목록을 전달하는 것이 아닌 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

In [58]:
from scipy.stats import uniform, randint

In [59]:
# 싸이파이의 stats 서브 패키지에 있는 uniform과 radint 클래스는 주어진 범위에서 고르게 값을 뽑는다. "균등 분포에서 샘플링한다."
# radint: 정숫값
# uniform: 실숫값
# 0-10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링

In [60]:
rgen = randint(0, 10)
rgen.rvs(10)

array([6, 3, 0, 2, 5, 7, 2, 8, 0, 6])

In [61]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104,  95, 100, 119, 107, 101,  93, 103,  86,  92]))

In [62]:
# 고르게 분포되는 것 같지 않지만, 샘플링 숫자를 키우면 각 숫자의 개수가 골고루 분포되는 것을 확인할 수 있다.

In [63]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.57857026, 0.27604889, 0.7687286 , 0.99812605, 0.29284455,
       0.23731072, 0.52826201, 0.63589984, 0.58552662, 0.33311651])

In [65]:
# uniform도 동일 난수 발생기랑 유사하게 생각 샘플링 횟수는 클수록 좋겠지?

In [67]:
# 이제 탐색할 매개변수의 딕셔너리를 생성
# min_samples_leaf 매개변수 추가 (리프 노드가 되기 위한 최소 샘플의 개수, 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다.)

In [68]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [69]:
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCVdml n_iter 매개변수에 지정

In [70]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15008a800>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x15b85b9a0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15008a6b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x150088730>},
                   random_state=42)

In [71]:
# 총 100번 샘플링 앞선 그리드 서치보다 검증수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다.

In [72]:
# 최적의 하이퍼파라미터 출력

In [73]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [74]:
# 최고의 교차 검증 점수 확인

In [75]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [76]:
# 최적의 모델은 전체 훈련세트로 훈련되어 저장되어 있음 해당 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인해보자.

In [77]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


In [78]:
# 테스트세트의 결과는 검증세트보다 약간 낮은 것이 일반적 괜찮은 결과이다.

In [79]:
# DecisionTreeClassifier 클래스에 splitter='random'을 추가하여 다시 훈련해보시오. 기본값은 best로 각 노드에서 최선의 분할을 찾는다.
# random이면 무작위로 분할한 다음 가장 좋은 것을 고른다.. 성능이 올라갔나요?

In [80]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15008a800>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x15b85b9a0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15008a6b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x150088730>},
                   random_state=42)

In [81]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


In [82]:
# 점수가 내려간 것을 확인할 수 있다.